<a href="https://colab.research.google.com/github/GustavoAdolfoGuizaWalteros/Deep_Learning/blob/main/P2_2_PLN_redes_neuronales_recurrentes_(RNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-gpu

In [2]:
import tensorflow as tf
import numpy as np
import os
import time
import sys

In [3]:
print("Version: ", tf.__version__)
#print("Eager mode: ", tf.executing_eagerly())
print("GPU esta", "disponible" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")
print("Dispositivos disponibles: ", tf.config.list_physical_devices())

Version:  2.7.0
GPU esta NOT AVAILABLE
Dispositivos disponibles:  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [4]:
#tf.device('/gpu:0') #activando la CPU
tf.device('/GPU:0') #activando la GPU 

In [6]:
fileDL= tf.keras.utils.get_file('ElGatoConBotas.txt','https://github.com/GustavoAdolfoGuizaWalteros/Deep_Learning/blob/main/Cuentos_txt/ElGatoConBotas.txt')
texto = open(fileDL, 'rb').read().decode(encoding='utf-8')

In [7]:
import re
from unicodedata import normalize
#pasa todo a minuscula
texto     = texto.lower()
#reemplazar tildes por letras similares sin tildes
transfor  = dict.fromkeys(map(ord, u'\u0301\u0308'), None)
texto     = normalize('NFKC', normalize('NFKD', texto).translate(transfor))
#quitar saltos de linea
texto      = texto.strip()
texto      = re.sub('\r|\n', ' ',texto)
#quitar espacios dobles
texto      = re.sub(' +', ' ', texto)
#quitando caracteres especiales
texto = re.sub(r"[^a-zA-Z0-9]+"," ",texto)
print(texto)

 doctype html html lang en data color mode auto data light theme light data dark theme dark head meta charset utf 8 link rel dns prefetch href https github githubassets com link rel dns prefetch href https avatars githubusercontent com link rel dns prefetch href https github cloud s3 amazonaws com link rel dns prefetch href https user images githubusercontent com link rel preconnect href https github githubassets com crossorigin link rel preconnect href https avatars githubusercontent com link crossorigin anonymous media all integrity sha512 1g4ryjktwrtqkn7fvfjuxh8rrzfujlgo77xmzfbfihzpx4bhvrzpe1vgnafttxi8g3y pywh3uxyhnkslp3 oa rel stylesheet href https github githubassets com assets light d46e2b60992dc114d02a7edf55f254c4 css link crossorigin anonymous media all integrity sha512 hi5b2oqte9njfjyrfuzxqa4bsgsnre5omc9iifhzy rdgg9qn4si1a97o0mlinlwft3xaifvolx0s7jhmhsvvw rel stylesheet href https github githubassets com assets dark 848e5bda8a9313d9e37e362b7eecd7a8 css link data color theme dar

In [8]:
print('el texto tiene longitud de:{} caracteres'. format(len(texto)))
vocab = sorted(set(texto))
print('el texto esta compuesto de estos :{} caracteres'. format(len(vocab)))
print(vocab)

el texto tiene longitud de:154693 caracteres
el texto esta compuesto de estos :37 caracteres
[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [9]:
char2idx = {u:i for i, u in enumerate(vocab)} # asignamos un número a cada vocablo
idx2char = np.array(vocab)
#-----------revisando las conversiones
#for char,_ in zip(char2idx, range(len(vocab))):
#    print(' {:4s}: {:3d},'.format(repr(char),char2idx[char]))

#pasamos todo el texto a números
texto_como_entero= np.array([char2idx[c] for c in texto])
print('texto: {}'.format(repr(texto[:100])))
print('{}'.format(repr(texto_como_entero[:100])))

texto: ' doctype html html lang en data color mode auto data light theme light data dark theme dark head met'
array([ 0, 14, 25, 13, 30, 35, 26, 15,  0, 18, 30, 23, 22,  0, 18, 30, 23,
       22,  0, 22, 11, 24, 17,  0, 15, 24,  0, 14, 11, 30, 11,  0, 13, 25,
       22, 25, 28,  0, 23, 25, 14, 15,  0, 11, 31, 30, 25,  0, 14, 11, 30,
       11,  0, 22, 19, 17, 18, 30,  0, 30, 18, 15, 23, 15,  0, 22, 19, 17,
       18, 30,  0, 14, 11, 30, 11,  0, 14, 11, 28, 21,  0, 30, 18, 15, 23,
       15,  0, 14, 11, 28, 21,  0, 18, 15, 11, 14,  0, 23, 15, 30])


In [10]:
import numpy as np

array_total = np.asarray([idx2char,texto_como_entero])

np.savetxt('sample.csv', array_total,fmt="%s", delimiter=",")

# Vocables to CSV
np.savetxt('vocables.csv', idx2char, fmt="%c", delimiter=",")

# Matriz Numerica to CSV
np.savetxt('matriz_numeros.csv', texto_como_entero,fmt="%s", delimiter=",")

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [11]:
char_dataset= tf.data.Dataset.from_tensor_slices(texto_como_entero)
#cantidad de secuencia de caracteres
secu_length=150
#creamos secuencias de maximo 100 caractereres
secuencias= char_dataset.batch(secu_length+1, drop_remainder=True)
for item in secuencias.take(10):
  print(repr(''.join(idx2char[item.numpy()])))

' doctype html html lang en data color mode auto data light theme light data dark theme dark head meta charset utf 8 link rel dns prefetch href https gi'
'thub githubassets com link rel dns prefetch href https avatars githubusercontent com link rel dns prefetch href https github cloud s3 amazonaws com lin'
'k rel dns prefetch href https user images githubusercontent com link rel preconnect href https github githubassets com crossorigin link rel preconnect '
'href https avatars githubusercontent com link crossorigin anonymous media all integrity sha512 1g4ryjktwrtqkn7fvfjuxh8rrzfujlgo77xmzfbfihzpx4bhvrzpe1vg'
'nafttxi8g3y pywh3uxyhnkslp3 oa rel stylesheet href https github githubassets com assets light d46e2b60992dc114d02a7edf55f254c4 css link crossorigin ano'
'nymous media all integrity sha512 hi5b2oqte9njfjyrfuzxqa4bsgsnre5omc9iifhzy rdgg9qn4si1a97o0mlinlwft3xaifvolx0s7jhmhsvvw rel stylesheet href https gith'
'ub githubassets com assets dark 848e5bda8a9313d9e37e362b7eecd7a8 css link d

In [12]:
#funcion para obtener el conjunto de datos de trainning
def split_input_target(chunk):
  input_text = chunk[:-1]
  target_text= chunk[1:]
  return input_text, target_text

dataset  = secuencias.map(split_input_target)
#el dataset contiene un conjunto de parejas de secuencia de texto
#(con la representación numérica de los caracteres), donde el 
#primer componente de la pareja contiene un paquete con una secuencia 
#de 100 caracteres del texto original y la segunda su correspondiente salida, 
#también de 100 caracteres. )
for input_example, target_example in dataset.take(1):
  print('input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print('Target data: ', repr(''.join(idx2char[target_example.numpy()])))

input data:  ' doctype html html lang en data color mode auto data light theme light data dark theme dark head meta charset utf 8 link rel dns prefetch href https g'
Target data:  'doctype html html lang en data color mode auto data light theme light data dark theme dark head meta charset utf 8 link rel dns prefetch href https gi'


In [14]:
#imprimimos el tensor del dataset
print(dataset)
#Hyper-Parametros para entrenamiento  de una rede neuronal 
#   -los datos se agrupan en batch
BATCH_SIZE= 64
#    -Tamaño de memoria disponible 
BUFFER_SIZE=10000
dataset= dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print (dataset)
#En el tensor dataset disponemos los datos de entrenamiento
#con agrupamienttos (batches) compuestos de 64 parejas de secuencias 
#de 100 integers de 64 bits que representan el carácter correspondiente 
#en el vocabulario.

<MapDataset shapes: ((150,), (150,)), types: (tf.int64, tf.int64)>
<BatchDataset shapes: ((64, 150), (64, 150)), types: (tf.int64, tf.int64)>


In [15]:
#como es un problema de clasificación estándar 
#para el que debemos definir la función de Lossy el optimizador.
def lossy(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

def create_model(vocab_size, embedding_dim, rnn_units, batch_size):
  #creando el modelo
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                         return_sequences=True,
                         stateful=True,
                         recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)                               
  ])
  #En cuanto al optimizador usaremos tf.keras.optimizers.Adam 
  #con los argumentos por defecto del optimizador Adam. 
  model.compile(optimizer='adam',
              loss=lossy,
              metrics=['accuracy'])
  return model
vocab_size= len(vocab)
#dimensiones de los vectores que tendrá la capa.
embedding_dim= 256
#cantidad de neuronas
rnn_units=1024
#creamos nuestra red neuronal RNN
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)
#summary()para visualizar la estructura del modelo
model.summary()
#resultados=  -La capa LSTM consta más de 5 millones de parametros)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           9472      
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 37)            37925     
                                                                 
Total params: 5,294,373
Trainable params: 5,294,373
Non-trainable params: 0
_________________________________________________________________


In [16]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
checkpoint_dir='/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/Redes neuronales recurrentes (RNN)/checkpointsV4/'
checkpoint_prefix= os.path.join(checkpoint_dir,"cp_{epoch:04d}.ckpt")


cp_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                               monitor='loss',
                                               verbose=1,
                                               save_weights_only=True,
                                               save_best_only=True,
                                               mode='auto')


In [18]:
EPOCHS=3
history=model.fit(dataset, 
                  epochs=EPOCHS, 
                  verbose=1,
                  callbacks=[cp_callback])

Epoch 1/3
16/16 [==============================] - ETA: 0s - loss: 3.3454 - accuracy: 0.1411 
Epoch 00001: loss improved from inf to 3.34538, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/Redes neuronales recurrentes (RNN)/checkpointsV4/cp_0001.ckpt
16/16 [==============================] - 210s 13s/step - loss: 3.3454 - accuracy: 0.1411
Epoch 2/3
16/16 [==============================] - ETA: 0s - loss: 3.1199 - accuracy: 0.1693 
Epoch 00002: loss improved from 3.34538 to 3.11988, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/Redes neuronales recurrentes (RNN)/checkpointsV4/cp_0002.ckpt
16/16 [==============================] - 205s 13s/step - loss: 3.1199 - accuracy: 0.1693
Epoch 3/3
16/16 [==============================] - ETA: 0s - loss: 2.9551 - accuracy: 0.2019 
Epoch 00003: loss improved from 3.11988 to 2.95511, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/Redes neuronales recurrentes (RNN)/checkpoint

In [19]:
#creamos un modelo con iguales caracteristicas al 1° modelo
model=create_model(vocab_size   =vocab_size,
                  embedding_dim =embedding_dim,
                  rnn_units     =rnn_units,
                  batch_size    =BATCH_SIZE)

#buscamos el ultimo checkpoint de entrenamiento
latest = tf.train.latest_checkpoint(checkpoint_dir)
print(latest)

/content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/Redes neuronales recurrentes (RNN)/checkpointsV4/cp_0003.ckpt


In [20]:
# cargamos los pesos al nuevo modelo (estos valores tienes una variación de un 10%)
model.load_weights(latest)
# continuamos el entrenamiento desde el checkpoint en que quedamos
history2=model.fit(dataset, 
                    epochs=3, 
                    verbose=1,
                    callbacks=[cp_callback])

Epoch 1/3
16/16 [==============================] - ETA: 0s - loss: 2.7219 - accuracy: 0.2584 
Epoch 00001: loss improved from 2.95511 to 2.72188, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/Redes neuronales recurrentes (RNN)/checkpointsV4/cp_0001.ckpt
16/16 [==============================] - 205s 13s/step - loss: 2.7219 - accuracy: 0.2584
Epoch 2/3
16/16 [==============================] - ETA: 0s - loss: 2.5620 - accuracy: 0.2882 
Epoch 00002: loss improved from 2.72188 to 2.56204, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/Redes neuronales recurrentes (RNN)/checkpointsV4/cp_0002.ckpt
16/16 [==============================] - 205s 13s/step - loss: 2.5620 - accuracy: 0.2882
Epoch 3/3
16/16 [==============================] - ETA: 0s - loss: 2.4175 - accuracy: 0.3087 
Epoch 00003: loss improved from 2.56204 to 2.41754, saving model to /content/gdrive/MyDrive/Colab Notebooks/P2_Deep_Learning/Redes neuronales recurrentes (RNN)/checkp

In [21]:
# You can change the directory name
LOG_DIR = 'tb_logs'

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

import os
if not os.path.exists(LOG_DIR):
  os.makedirs(LOG_DIR)
  
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR))

get_ipython().system_raw('./ngrok http 6006 &')

!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2021-11-22 22:17:43--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.3’

ngrok-stable-linux- 100%[===================>]  13.19M  3.28MB/s    in 5.0s    

2021-11-22 22:17:48 (2.61 MB/s) - ‘ngrok-stable-linux-amd64.zip.3’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
https://4ea8-34-83-45-70.ngrok.io


In [22]:
tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=LOG_DIR, 
                         histogram_freq=1,
                         write_graph=True,
                         write_grads=True,
                         batch_size=BATCH_SIZE,
                         write_images=True)

In [23]:
#model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)
history_TB=model.fit(dataset, 
                    epochs=4, 
                    verbose=1,
                    callbacks=[tbCallBack])

Epoch 1/4
16/16 [==============================] - 210s 13s/step - loss: 2.2857 - accuracy: 0.3378
Epoch 2/4
16/16 [==============================] - 222s 14s/step - loss: 2.1491 - accuracy: 0.3839
Epoch 3/4
16/16 [==============================] - 221s 14s/step - loss: 2.0066 - accuracy: 0.4335
Epoch 4/4
16/16 [==============================] - 223s 14s/step - loss: 1.8304 - accuracy: 0.4950


In [24]:
#creamos un modelo tomando como base el ultimo checkpoint
model = create_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1,None]))

In [25]:
#funcion para generar texto
def generate_text(model, start_string):
  #definimos cuantos tensores/cantidad de texto generaremos
  num_generate=500
  #convertimos el texto en números
  input_eval=[char2idx[s] for s in start_string]
  input_eval= tf.expand_dims (input_eval,0)
  text_generated = []

  temperature = 0.2  #(0.0 a  1) entre más alta la temperatura más creatividad al modelo, pero tambien más errores ortograficos.
  model.reset_states() #bucle para generar caracteres, mediante predicciones
  for i in range(num_generate):
    predictions = model(input_eval)
    predictions = tf.squeeze(predictions, 0)
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
    input_eval= tf.expand_dims([predicted_id],0)
    text_generated.append (idx2char[predicted_id])
  
  return (start_string+ ''.join(text_generated))

In [26]:
print(generate_text(model, start_string=u"un molindero"))

un molindero data lime de de de de cot huot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quot quo


In [36]:
from keras.models import model_from_json
import os
dir_export= '/content/gdrive/MyDrive/Colab Notebooks/Modelos'
#dir_export= os.path.join(dir_drive)
# Serializamos el modelo en forma JSON
model_json = model.to_json()
with open(os.path.join(dir_export,'RNN_ElGatoConBotas_json.json'), 'w') as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join(dir_export,'RNN_ElGatoConBotas_pesos.hdf5'))
model.save(os.path.join(dir_export,'RNN_ElGatoConBotas_model.h5'))
print("modelo salvado en Drive de google")

modelo salvado en Drive de google


In [37]:
!wget https://github.com/GustavoAdolfoGuizaWalteros/Deep_Learning/blob/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5?raw=true \
      -O RNN_ElGatoConBotas_model.h5

--2021-11-22 22:51:50--  https://github.com/GustavoAdolfoGuizaWalteros/Deep_Learning/blob/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5?raw=true
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/GustavoAdolfoGuizaWalteros/Deep_Learning/raw/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5 [following]
--2021-11-22 22:51:50--  https://github.com/GustavoAdolfoGuizaWalteros/Deep_Learning/raw/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/GustavoAdolfoGuizaWalteros/Deep_Learning/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5 [following]
--2021-11-22 22:51:50--  https://raw.githubusercontent.com/GustavoAdolfoGuizaWalteros/Deep_Learning/main/Modelos/RNN_ElGatoConBotas_pesos.hdf5
Resolving raw.githubusercontent.com (raw.gi

In [38]:
!pip install pyprind

In [ ]:
def reporthook(count, block_size, total_size):
    global start_time
    if count == 0:
        start_time = time.time()
        return
    duration = time.time() - start_time
    progress_size = int(count * block_size)
    speed = progress_size / (1024.**2 * duration)
    percent = count * block_size * 100. / total_size
    sys.stdout.write("\r%d%% | %d MB | %.2f MB/s | %d segundos transcurrido" %
                    (percent, progress_size / (1024.**2), speed, duration))
    sys.stdout.flush()

import urllib.request
url_github_Model='https://github.com/luisFernandoCastellanosG/Machine_learning/blob/master/DeepLearning/PLN/recurrent_network_RNN/Modelos/RNN_LasMinasDelReySalomon_model.h5?raw=true'
urllib.request.urlretrieve(url_github_Model,
                           'RNN_ElGatoConBotas_model.h5', 
                           reporthook)

In [ ]:
new_model = tf.keras.models.load_model('/content/gdrive/MyDrive/Colab Notebooks/Modelos/RNN_ElGatoConBotas_model.h5')

In [ ]:
print(generate_text(new_model, start_string=u"los fantasmas de "))